System information (for reproducibility):

In [1]:
versioninfo()

Julia Version 1.10.2
Commit bd47eca2c8a (2024-03-01 10:14 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: macOS (x86_64-apple-darwin22.4.0)
  CPU: 11 × Apple M3 Pro
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, westmere)
Threads: 1 default, 0 interactive, 1 GC (on 11 virtual cores)


Load packages:

In [2]:
using Pkg

Pkg.activate(pwd())
Pkg.instantiate()
Pkg.status()

  Activating project at `~/Documents/GitHub/biostat-m257-2024-spring/hw4`


Status `~/Documents/GitHub/biostat-m257-2024-spring/hw4/Project.toml`
  [6e4b80f9] BenchmarkTools v1.5.0
  [944b1d66] CodecZlib v0.7.4
  [8bb1440f] DelimitedFiles v1.9.1
  [42fd0dbc] IterativeSolvers v0.9.4
  [0b1a1467] KrylovKit v0.7.1
  [bdcacae8] LoopVectorization v0.12.170
  [b51810bb] MatrixDepot v1.0.11
  [f0f68f2c] PlotlyJS v0.18.13
  [91a5bcdd] Plots v1.40.4
  [295af30f] Revise v3.5.14
  [2913bbd2] StatsBase v0.34.3
  [b8865327] UnicodePlots v3.6.4
  [0f1e0344] WebIO v0.8.21
  [37e2e46d] LinearAlgebra
  [9a3f8284] Random
  [2f01184e] SparseArrays v1.10.0


We are going to try different numerical methods learnt in class on the [Google PageRank problem](https://en.wikipedia.org/wiki/PageRank).

## Q1 (5 pts) Recognize structure

Let $\mathbf{A} \in \{0,1\}^{n \times n}$ be the connectivity matrix of $n$ web pages with entries
$$
\begin{eqnarray*}
	a_{ij}= \begin{cases}
	1 & \text{if page $i$ links to page $j$} \\
	0 & \text{otherwise}
	\end{cases}.
\end{eqnarray*}
$$
$r_i = \sum_j a_{ij}$ is the out-degree of page $i$. That is $r_i$ is the number of links on page $i$. Imagine a random surfer exploring the space of $n$ pages according to the following rules.  

- From a page $i$ with $r_i>0$
    * with probability $p$, (s)he randomly chooses a link on page $i$ (uniformly) and follows that link to the next page  
    * with probability $1-p$, (s)he randomly chooses one page from the set of all $n$ pages (uniformly) and proceeds to that page  
- From a page $i$ with $r_i=0$ (a dangling page), (s)he randomly chooses one page from the set of all $n$ pages (uniformly) and proceeds to that page  
    
The process defines a Markov chain on the space of $n$ pages. Write the transition matrix $\mathbf{P}$ of the Markov chain as a sparse matrix plus rank 1 matrix.

### Solution

$$
\begin{eqnarray*}
	p_{ij}= \begin{cases}
	\frac{1-p}{n} + \frac{pa_{ij}}{r_i} & \text{if $r_i>0$} \\
	\frac{1}{n} & \text{if $r_i = 0$}
	\end{cases}.
\end{eqnarray*}
$$

$$
\mathbf{P} = \frac{1}{n}*\begin{bmatrix}(1-p)^{\delta_{r_1}} \\ (1-p)^{\delta_{r_2}} \\ \vdots \\ (1-p)^{\delta_{r_n}} \end{bmatrix}\begin{bmatrix}1 & 1 & \cdots & 1 \end{bmatrix} + p * diag\begin{bmatrix} d_1 & d_2 & \cdots & d_n \end{bmatrix} \mathbf{A}
$$

where $\delta_{r_i} = \begin{cases}
	1 & \text{if $r_i>0$} \\
	0 & \text{if $r_i = 0$}
	\end{cases}$, $d_i = \begin{cases}
	1/r_i & \text{if $r_i>0$} \\
	0 & \text{if $r_i = 0$}
	\end{cases}$

## Q2 Relate to numerical linear algebra

According to standard Markov chain theory, the (random) position of the surfer converges to the stationary distribution $\mathbf{x} = (x_1,\ldots,x_n)^T$ of the Markov chain. $x_i$ has the natural interpretation of the proportion of times the surfer visits page $i$ in the long run. Therefore $\mathbf{x}$ serves as page ranks: a higher $x_i$ means page $i$ is more visited. It is well-known that $\mathbf{x}$ is the left eigenvector corresponding to the top eigenvalue 1 of the transition matrix $\mathbf{P}$. That is $\mathbf{P}^T \mathbf{x} = \mathbf{x}$. Therefore $\mathbf{x}$ can be solved as an **eigen-problem**. It can also be cast as **solving a linear system**. Since the row sums of $\mathbf{P}$ are 1, $\mathbf{P}$ is rank deficient. We can replace the first equation by the $\sum_{i=1}^n x_i = 1$.

Hint: For iterative solvers, we don't need to replace the 1st equation. We can use the matrix $\mathbf{I} - \mathbf{P}^T$ directly if we start with a vector with all positive entries.

## Q3 (10 pts) Explore data

Obtain the connectivity matrix `A` from the `SNAP/web-Google` data in the MatrixDepot package. 

In [3]:
using MatrixDepot
md = mdopen("SNAP/web-Google")
# display documentation for the SNAP/web-Google data
mdinfo(md)

[ Info: verify download of index files...
[ Info: reading database
[ Info: adding metadata...
[ Info: adding svd data...
[ Info: writing database
[ Info: used remote sites are sparse.tamu.edu with MAT index and math.nist.gov with HTML index


# SNAP/web-Google

###### MatrixMarket matrix coordinate pattern general

---

  * UF Sparse Matrix Collection, Tim Davis
  * http://www.cise.ufl.edu/research/sparse/matrices/SNAP/web-Google
  * name: SNAP/web-Google
  * [Web graph from Google]
  * id: 2301
  * date: 2002
  * author: Google
  * ed: J. Leskovec
  * fields: name title A id date author ed kind notes
  * kind: directed graph

---

  * notes:
  * Networks from SNAP (Stanford Network Analysis Platform) Network Data Sets,
  * Jure Leskovec http://snap.stanford.edu/data/index.html
  * email jure at cs.stanford.edu
  * 
  * Google web graph
  * 
  * Dataset information
  * 
  * Nodes represent web pages and directed edges represent hyperlinks between them.
  * The data was released in 2002 by Google as a part of Google Programming
  * Contest.
  * 
  * Dataset statistics
  * Nodes   875713
  * Edges   5105039
  * Nodes in largest WCC    855802 (0.977)
  * Edges in largest WCC    5066842 (0.993)
  * Nodes in largest SCC    434818 (0.497)
  * Edges in largest SCC    3419124 (0.670)
  * Average clustering coefficient  0.6047
  * Number of triangles     13391903
  * Fraction of closed triangles    0.05523
  * Diameter (longest shortest path)    22
  * 90-percentile effective diameter    8.1
  * 
  * Source (citation)
  * 
  * J. Leskovec, K. Lang, A. Dasgupta, M. Mahoney. Community Structure in Large
  * Networks: Natural Cluster Sizes and the Absence of Large Well-Defined Clusters.
  * arXiv.org:0810.1355, 2008.
  * 
  * Google programming contest, 2002
  * http://www.google.com/programming-contest/
  * 
  * Files
  * File    Description
  * web-Google.txt.gz   Webgraph from the Google programming contest, 2002

---

916428 916428 5105039


In [4]:
# connectivity matrix
A = md.A

916428×916428 SparseArrays.SparseMatrixCSC{Bool, Int64} with 5105039 stored entries:
⎡⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎤
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎣⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎦

In [5]:
using UnicodePlots
spy(A)

           ┌──────────────────────────────────────────┐    
         1 │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│ > 0
           │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│ < 0
           │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
           │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
           │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
           │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
           │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
           │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
           │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
           │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
           │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
           │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
           │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
           │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
           │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
           │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿

Compute summary statistics:  

* How much memory does `A` take? If converted to a `Matrix{Float64}` (don't do it!), how much memory will it take?  

* number of web pages

* number of edges (web links)

* number of dangling nodes (pages with no out links)

* histogram of in-degrees  

* list the top 20 pages with the largest in-degrees?  

* histogram of out-degrees

* which the top 20 pages with the largest out-degrees?

* visualize the sparsity pattern of $\mathbf{A}$ or a submatrix of $\mathbf{A}$ say `A[1:10000, 1:10000]`. 

### Solution 

1.

In [6]:
Base.summarysize(A)

53276943

Now we can see that the current matrix $\mathbf{A}$ takes 53276943 bits of storage. If we convert it to the form `Matrix{Float64}`, then it will take 916428 * 916428 * 8 = 6718722233472 = 6.11 TB (divided by $1024^4$) to storage.

2.

There are totally 916428 webpages contained in this dataset.

3.

In [7]:
sum(A)

5105039

There are totally 5105039 web links.

4.

In [8]:
r = sum(A, dims = 2)
sum(r .== 0)

176974

There are totally 176974 dangling nodes.

5.

In [9]:
r_in = sum(A, dims = 1)

1×916428 Matrix{Int64}:
 212  6  44  3  5  15  0  3  1  1  13  1  …  17  7  14  2  0  14  1  0  44  2

In [10]:
using UnicodePlots
UnicodePlots.histogram(r_in)

                    ┌                                        ┐ 
   [   0.0,  500.0) ┤███████████████████████████████  916 114  
   [ 500.0, 1000.0) ┤▏ 180                                     
   [1000.0, 1500.0) ┤▏ 32                                      
   [1500.0, 2000.0) ┤▏ 20                                      
   [2000.0, 2500.0) ┤▏ 16                                      
   [2500.0, 3000.0) ┤▏ 20                                      
   [3000.0, 3500.0) ┤▏ 18                                      
   [3500.0, 4000.0) ┤▏ 10                                      
   [4000.0, 4500.0) ┤▏ 9                                       
   [4500.0, 5000.0) ┤▏ 4                                       
   [5000.0, 5500.0) ┤▏ 4                                       
   [5500.0, 6000.0) ┤  0                                       
   [6000.0, 6500.0) ┤▏ 1                                       
                    └                                        ┘ 
                                     Fre

In [11]:
using StatsBase
r_in_table = countmap(r_in)
sort(r_in_table, byvalue = false, rev = false)

OrderedCollections.OrderedDict{Int64, Int64} with 705 entries:
  0  => 201883
  1  => 286895
  2  => 114542
  3  => 58863
  4  => 36633
  5  => 27869
  6  => 23752
  7  => 19255
  8  => 15987
  9  => 14154
  10 => 11523
  11 => 10037
  12 => 8739
  13 => 7552
  14 => 6768
  15 => 6207
  16 => 5726
  17 => 5230
  18 => 4858
  19 => 4230
  20 => 3744
  21 => 2859
  22 => 2234
  23 => 2001
  24 => 1885
  ⋮  => ⋮

6.

In [12]:
r_in_sort = sortperm(r_in, dims = 2, rev = true)
collect(zip(r_in_sort, r_in[r_in_sort]))[1:20]

20-element Vector{Tuple{Int64, Int64}}:
 (537040, 6326)
 (597622, 5354)
 (504141, 5271)
 (751385, 5182)
 (32164, 5097)
 (885606, 4847)
 (163076, 4731)
 (819224, 4620)
 (605857, 4550)
 (828964, 4484)
 (551830, 4220)
 (41910, 4219)
 (558792, 4206)
 (459075, 4187)
 (407611, 4180)
 (213433, 4084)
 (765335, 4015)
 (384667, 4010)
 (173977, 3988)
 (687326, 3956)

The top 20 page indices (left) with the largest in-degrees (right) are shown above.

7.

In [13]:
r_out = sum(A, dims = 2)
UnicodePlots.histogram(r_out)

                  ┌                                        ┐ 
   [  0.0,  20.0) ┤███████████████████████████████  891 798  
   [ 20.0,  40.0) ┤▊ 22 628                                  
   [ 40.0,  60.0) ┤▏ 1 329                                   
   [ 60.0,  80.0) ┤▏ 371                                     
   [ 80.0, 100.0) ┤▏ 124                                     
   [100.0, 120.0) ┤▏ 86                                      
   [120.0, 140.0) ┤▏ 29                                      
   [140.0, 160.0) ┤▏ 15                                      
   [160.0, 180.0) ┤▏ 15                                      
   [180.0, 200.0) ┤▏ 8                                       
   [200.0, 220.0) ┤▏ 7                                       
   [220.0, 240.0) ┤▏ 3                                       
   [240.0, 260.0) ┤▏ 8                                       
   [260.0, 280.0) ┤▏ 3                                       
   [280.0, 300.0) ┤  0                                       
   [300.

In [14]:
r_out_table = countmap(r_out)
sort(r_out_table, byvalue = false, rev = false)

OrderedCollections.OrderedDict{Int64, Int64} with 189 entries:
  0  => 176974
  1  => 127937
  2  => 95253
  3  => 65976
  4  => 53560
  5  => 46771
  6  => 40993
  7  => 38491
  8  => 37187
  9  => 32666
  10 => 29096
  11 => 25656
  12 => 22549
  13 => 19156
  14 => 17018
  15 => 15130
  16 => 13070
  17 => 11778
  18 => 11281
  19 => 11256
  20 => 8305
  21 => 3424
  22 => 2272
  23 => 1645
  24 => 1207
  ⋮  => ⋮

8.

In [15]:
r_out_sort = sortperm(r_out, dims = 1, rev = true)
collect(zip(r_out_sort, r_out[r_out_sort]))[1:20]

20-element Vector{Tuple{Int64, Int64}}:
 (506743, 456)
 (203749, 372)
 (305230, 372)
 (768092, 330)
 (808644, 277)
 (412411, 268)
 (600480, 265)
 (376429, 258)
 (156951, 257)
 (885729, 256)
 (667585, 253)
 (685696, 248)
 (282141, 247)
 (598189, 245)
 (579315, 244)
 (411594, 231)
 (321092, 229)
 (838279, 225)
 (302734, 216)
 (915274, 213)

The top 20 page indices (left) with the largest out-degrees (right) are shown above.

9.

In [16]:
using SparseArrays
spy(A[1 : 10000, 1 : 10000])

          ┌──────────────────────────────────────────┐    
        1 │⠀⠀⠀⠀⠀⠀⠀⠂⠀⠀⠀⠀⢀⢂⡂⠐⠀⠀⠀⠠⠀⠀⠀⠔⠄⠀⠈⢀⠀⠀⠉⠀⠀⠁⠀⠀⠐⠀⠄⠀⠐⠀│ > 0
          │⠀⠀⠀⠀⠀⠂⡂⠀⠄⠄⠈⠈⡔⠀⠀⠁⠂⠀⠀⠀⠀⠠⠁⠒⠁⠀⠄⠀⠐⠀⠈⠀⠀⡐⠀⠀⠠⠀⡀⠀⠉⠀│ < 0
          │⠀⠐⠠⠀⠀⠁⢐⠄⠀⠢⠠⠀⠀⡀⠈⠀⠋⠀⠀⠀⠀⠀⠂⡀⠀⠈⠄⠀⠄⠁⠈⠀⠠⠀⠰⠀⠀⠀⠀⢀⠆⠀│    
          │⠀⠈⠠⠂⠀⠈⠀⡠⠐⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡀⠈⠄⠁⢀⡀⠐⠄⠄⠀⠀⢀⠀⠈⠂⢠⠀⠢⠐⠀⠀⠀⡀│    
          │⠀⠀⠐⡁⠀⡀⠀⠀⠠⠀⠁⠀⠀⠀⡄⠀⠈⠁⠀⠀⠀⠀⠠⠂⠀⠀⠀⠀⠀⠂⠪⠀⠀⠂⠀⠀⠀⠄⢀⠀⠁⠀│    
          │⠂⢀⠀⠁⠀⠀⡀⡉⠀⠀⠠⠂⠐⢀⠄⠁⠐⠀⠈⠀⠀⠀⠀⠀⡀⠐⠁⡠⠀⠂⠚⠀⠁⠀⠐⢀⠆⠀⠈⠄⢀⠂│    
          │⠀⠀⠀⠁⠀⡄⠀⠀⠀⠀⠒⠀⢄⠄⢂⠀⠂⠀⠀⠄⠀⢀⠀⠣⠀⢀⠀⠀⠀⠀⠠⡀⠀⠀⢠⠄⠀⡀⠀⠂⠂⠀│    
          │⢀⠈⠀⠀⠂⠀⠀⠁⢠⠈⠄⠁⠀⠀⡈⠈⠠⠀⢀⡦⠀⠀⠅⠐⠀⠨⠁⠨⠀⠄⠠⢀⠀⠐⢐⠀⠀⠉⠀⠀⠠⠉│    
          │⠀⠀⠨⠀⠋⠀⠄⠀⠂⠀⠈⠀⠀⠀⠈⠀⠀⠀⠀⡑⠀⢀⠈⠈⠀⠀⠀⠈⠀⠂⠐⠼⠀⠠⠀⠀⡈⠁⠀⠀⠡⠄│    
          │⠠⡁⠀⠀⠀⠀⠀⠀⠀⠀⠂⢀⠀⠀⠀⡄⠀⠀⠀⠤⠀⠀⠁⠀⡂⠀⠀⠀⠄⠀⢈⠀⢔⡊⣨⠈⠂⠀⠄⢀⠀⠀│    
          │⠀⠂⠀⠈⠀⠀⠀⠀⠀⠀⠈⠀⡀⠀⠂⠠⠐⠀⠄⠀⠀⠀⠀⠀⠀⠤⠀⠀⠀⠠⠘⠐⠀⠄⢘⠀⠀⢐⠄⠀⠁⠀│    
          │⢀⠄⢠⠄⡊⠂⠢⢀⠀⠀⠀⠀⠄⡀⡑⠀⡐⠀⠁⡂⠀⠀⠀⠰⠂⢠⠄⠈⠀⠈⠐⠀⠀⠀⠉⠀⠀⡀⠀⠀⠃⠀│    
          │⠀⠀⠅⠀⡁⠀⠀⠀⠀⠀⢁⢀⠀⡀⡀⡐⠀⠀⠈⠈⠔⠄⠀⠀⢀⠀⠀⠀⠀⠀⠀⠀⠀⠄⢉⠠⠁⠈⠀⠀⠀⠀│    
          │⠊⢈⠀⠁⠀⠀⠀⠅⠀⠂⠀⡀⠀⠀⠀⠀⡀⠀⠀⠐⠄⠁⠠⠀⠐⠈⠀⠀⠀⡀⢠⠀⠀⠀⢰⠀⠀⠀⢀⠀⠠⠂│    
          │⠈⠀⠀⠈⠄⠀⠀⠄⠐⠀⠠⠀⠁⠀⡂⠀⠠⠄⠌⠅⣀⠀⠀⢀⠀⠠⠀⠠⠠⠀⠲⢂⠀⠰⡄⠀⢀⠬⠈⠀⠀⠀│    
          │⠈⠀⠀⠄⠀⠀⡀⠈⠀⠀⠙⠀⠀⠀⢀⢀⠂⠀⠀⡄⢀⠀⠠⠀⠂⠀⠀⠐⠀⠀⠸⠀⠀⠀⠈⠠⠀⠠⠁⠀⠂⠀│  

Hence the sparsity of this submatrix is $600/10000^2 = 6\times 10^{-6}$.

## Q4 (5 pts) Dense linear algebra? 

Consider the following methods to obtain the page ranks of the `SNAP/web-Google` data. 

1. A dense linear system solver such as LU decomposition.  
2. A dense eigen-solver for asymmetric matrix.  

For the LU approach, estimate (1) the memory usage and (2) how long it will take assuming that the LAPACK functions can achieve the theoretical throughput of your computer. 

### Solution 

1. The memory usage.
As LU decomposition is going to use a unit lower triangular matrix $\mathbf{L}$ and an upper triangular matrix $\mathbf{U}$ to represent the original matrix $\mathbf{A}$, it will take the storage space calculated in Q3.1 plus an additional diagonal matrix, whose sum is approximately 6.11 TB.
2. The time consuming.
From lecture notes, we have already shown that LU decomposition costs approximate $\frac{2}{3}n^3$ flops, which implies here it will costs $\frac{2}{3}\times 916428^3 = 5.13\times 10^{17}$ flops. My computer can do 11 cores * 4.05 GHz * 16 FLOP/cycle = 712.8 GFLOPS per second. Then we use total flops divided by my computing power and get the result that I need $7.2 \times 10^5$ seconds (approximately 199.9 days).

## Q5 (75 pts) Iterative solvers

Set the _teleportation_ parameter at $p = 0.85$. Consider the following methods for solving the PageRank problem. 

1. An iterative linear system solver such as GMRES. 
2. An iterative eigen-solver such as Arnoldi method.

For iterative methods, we have many choices in Julia. See a list of existing Julia packages for linear solvers at this [page](https://jutho.github.io/KrylovKit.jl/stable/#Package-features-and-alternatives-1). The start-up code below uses the [KrylovKit.jl](https://github.com/Jutho/KrylovKit.jl) package. You can use other packages if you prefer. Make sure to utilize the special structure of $\mathbf{P}$ (sparse + rank 1) to speed up the matrix-vector multiplication. 

### Step 1 (15 pts)

Let's implement a type `PageRankImPt` that mimics the matrix $\mathbf{M} = \mathbf{I} - \mathbf{P}^T$. For iterative methods, all we need to provide are methods for evaluating $\mathbf{M} \mathbf{v}$ and $\mathbf{M}^T \mathbf{v}$ for arbitrary vector $\mathbf{v}$.

In [17]:
using BenchmarkTools, LinearAlgebra, Random, SparseArrays, Revise

In [18]:
# a type for the matrix M = I - P^T in PageRank problem
struct PageRankImPt{TA <: Number, IA <: Integer, T <: AbstractFloat} <: AbstractMatrix{T}
    A         :: SparseMatrixCSC{TA, IA} # adjacency matrix
    telep     :: T
    r         :: Vector{IA}
    rinv      :: Vector{T}
    z         :: Vector{T}
    rinvA     :: SparseMatrixCSC{T, IA}
    rinvAT    :: Transpose{T, SparseMatrixCSC{T, IA}}
    storagez  :: Vector{T}
    # working arrays
    # TODO: whatever intermediate arrays you may want to pre-allocate
end

# constructor
function PageRankImPt(A::SparseMatrixCSC, telep::T) where T <: AbstractFloat
    n = size(A, 1)
    # TODO: initialize and pre-allocate arrays
    r = vec(sum(A, dims = 2)) #916428×1 vector
    rinv = vec(ifelse.(r .> 0, telep ./ r, 0.)) #916428×1 vector
    z = vec(max.((1 - telep), (1 .- r)) ./ n) #916428×1 vector
    rinvA = Diagonal(vec(rinv)) * A
    rinvAT = transpose(rinvA)
    storagez = similar(z) #916428×1 vector, used to storage z * (1,1,...,1) * v
    PageRankImPt(A, telep, r, rinv, z, rinvA, rinvAT, storagez)
end

# matrix element access M[i, j]
Base.getindex(M::PageRankImPt, i, j) = (i == j) - M.rinv[j] * M.A[j, i] - M.z[j]
LinearAlgebra.issymmetric(::PageRankImPt) = false
Base.size(M::PageRankImPt) = size(M.A)

# overwrite `out` by `(I - Pt) * v`, i.e. 'M * v'
function LinearAlgebra.mul!(
        out :: Vector{T}, 
        M   :: PageRankImPt{<:Number, <:Integer, T}, 
        v   :: Vector{T}
        ) where T <: AbstractFloat
    # TODO: implement mul!(out, M, v)
    mul!(out, M.rinvAT, v)
    out .= v .- out .- dot(M.z, v)
    return out
end

# overwrite `out` by `(I - P) * v`, i.e. 'Mt * v'
function LinearAlgebra.mul!(
        out :: Vector{T}, 
        Mt  :: Transpose{T, PageRankImPt{TA, IA, T}}, 
        v   :: Vector{T}
        ) where {TA<:Number, IA<:Integer, T <: AbstractFloat}
    M = Mt.parent
    # TODO: implement mul!(out, transpose(M), v)
    mul!(out, M.rinvA, v)
    M.storagez .= sum(v) .* M.z
    out .= v .- out .- M.storagez
    return out
end

To check correctness. Note 
$$
\mathbf{M}^T \mathbf{1} = \mathbf{0}
$$
and
$$
\mathbf{M} \mathbf{x} = \mathbf{0}
$$
for stationary distribution $\mathbf{x}$.

Download the solution file `pgrksol.csv.gz`. **Do not put this file in your Git**. You will lose points if you do. You can add a line `pgrksol.csv.gz` to your `.gitignore` file.

In [19]:
using CodecZlib, DelimitedFiles

isfile("pgrksol.csv.gz") || download("https://github.com/ucla-biostat-257/2024spring/raw/master/hw/hw4/pgrksol.csv.gz")
xsol = open("pgrksol.csv.gz", "r") do io
    vec(readdlm(GzipDecompressorStream(io)))
end

916428-element Vector{Float64}:
 3.3783428216975054e-5
 2.0710155392568165e-6
 3.663065984832893e-6
 7.527510785028837e-7
 8.63328599674051e-7
 1.769418252415541e-6
 2.431230382883396e-7
 6.368417180141445e-7
 4.744973703681939e-7
 2.6895486110647536e-7
 3.18574314847409e-6
 7.375106374416742e-7
 2.431230382883396e-7
 ⋮
 1.1305006040148547e-6
 4.874825281822915e-6
 3.167946973112519e-6
 9.72688040308568e-7
 6.588614479285245e-7
 7.737011774300648e-7
 2.431230382883396e-7
 1.6219204214797293e-6
 3.912130060551738e-7
 2.431230382883396e-7
 7.296033831163157e-6
 6.330939996912478e-7

**You will lose all 35 points (Steps 1 and 2)** if the following statements throw AssertError.

In [20]:
M = PageRankImPt(A, 0.85)
n = size(M, 1)

#@assert transpose(M) * ones(n) ≈ zeros(n)
@assert norm(transpose(M) * ones(n)) < 1e-12

In [21]:
#@assert M * xsol ≈ zeros(n)
@assert norm(M * xsol) < 1e-12

### Step 2 (20 pts)

We want to benchmark the hot functions `mul!` to make sure they are efficient and allocate no memory.

In [22]:
M = PageRankImPt(A, 0.85)
n = size(M, 1)
v, out = ones(n), zeros(n)
bm_mv = @benchmark mul!($out, $M, $v) setup=(fill!(out, 0); fill!(v, 1))

BenchmarkTools.Trial: 391 samples with 1 evaluation.
 Range (min … max):  11.324 ms … 13.098 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     12.546 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   12.560 ms ± 91.768 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

                                                  ██▂▁         
  ▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃█████▇▄▄▃▃▁▂ ▂
  11.3 ms         Histogram: frequency by time        12.8 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [23]:
bm_mtv = @benchmark mul!($out, $(transpose(M)), $v) setup=(fill!(out, 0); fill!(v, 1))

BenchmarkTools.Trial: 436 samples with 1 evaluation.
 Range (min … max):  11.117 ms … 11.807 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     11.265 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   11.268 ms ± 60.480 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

                     ▄ ▃▃ ▁   ▃▂▃█▃▃▂▂▃▂▂▂ ▄                   
  ▄▃▁▁▁▃▁▃▃▃▃▃▆▁▆▄▄▇▆████████▇████████████▇██▇▆▄▄█▄▃▄▅▁▄▃▁▁▃▃ ▄
  11.1 ms         Histogram: frequency by time        11.4 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

You will lose 1 point for each 100 bytes memory allocation. So the points you will get is

In [24]:
clamp(10 - median(bm_mv).memory / 100, 0, 10) + 
clamp(10 - median(bm_mtv).memory / 100, 0, 10)

20.0

**Hint**: My median run times are about 10 ms and memory allocations are 0 bytes.

### Step 3 (20 pts)

Let's first try to solve the PageRank problem by the GMRES method for solving linear equations. 

In [25]:
using KrylovKit

# normalize in-degrees to be the start point
x0   = vec(sum(A, dims = 1)) .+ 1.0
x0 ./= sum(x0)

# right hand side
b = zeros(n)

# warm up (compilation)
linsolve(M, b, x0, issymmetric = false, isposdef = false, maxiter = 1) 
# output is complex eigenvalue/eigenvector
(x_gmres, info), time_gmres, = @timed linsolve(M, b, x0, issymmetric = false, isposdef = false)

(value = ([3.378342822196307e-5, 2.0710155392529946e-6, 3.6630659852452833e-6, 7.52751078561982e-7, 8.633285997180924e-7, 1.7694182527417869e-6, 2.431230382917873e-7, 6.368417180761666e-7, 4.744973703761079e-7, 2.6895486111028844e-7  …  3.1679469739846743e-6, 9.726880410252835e-7, 6.588614478557539e-7, 7.737011774732227e-7, 2.431230382917873e-7, 1.6219204214282731e-6, 3.9121300606405074e-7, 2.431230382917873e-7, 7.296033831341722e-6, 6.330939996693129e-7], ConvergenceInfo: one converged value after 3 iterations and 72 applications of the linear map;
norms of residuals are given by (7.821153784931289e-13,).
), time = 5.414369041, bytes = 1511656912, gctime = 0.112910666, gcstats = Base.GC_Diff(1511656912, 206, 0, 18357, 4, 187, 112910666, 4, 1))

Check correctness. **You will lose all 20 points if the following statement throws `AssertError`.**

In [26]:
@assert norm(x_gmres - xsol) < 1e-8

GMRES should be reasonably fast. The points you'll get is

In [27]:
clamp(20 / time_gmres * 20, 0, 20)

20.0

**Hint**: My runtime is about 3-4 seconds.

### Step 4 (20 pts)

Let's first try to solve the PageRank problem by the Arnoldi method for solving eigen problems. 

In [28]:
# warm up (compilation)
eigsolve(M, x0, 1, :SR, issymmetric = false, maxiter = 1)
# output is complex eigenvalue/eigenvector
(vals, vecs, info), time_arnoldi, = @timed eigsolve(M, x0, 1, :SR, issymmetric = false)

(value = (ComplexF64[-2.900275677874062e-14 + 0.0im], Vector{ComplexF64}[[0.005635826953807274 + 0.0im, 0.000345491438078285 + 0.0im, 0.0006110808494125225 + 0.0im, 0.00012557561626035956 + 0.0im, 0.00014402240532803974 + 0.0im, 0.00029517830503985806 + 0.0im, 4.055832828678389e-5 + 0.0im, 0.00010623935784852585 + 0.0im, 7.915671116196185e-5 + 0.0im, 4.486765066725453e-5 + 0.0im  …  0.0005284839899796965 + 0.0im, 0.0001622659914798873 + 0.0im, 0.00010991273837610376 + 0.0im, 0.0001290705585575232 + 0.0im, 4.055832828678389e-5 + 0.0im, 0.0002705723874320486 + 0.0im, 6.52630274833394e-5 + 0.0im, 4.055832828678389e-5 + 0.0im, 0.001217140660132899 + 0.0im, 0.0001056141551070012 + 0.0im]], ConvergenceInfo: one converged value after 7 iterations and 99 applications of the linear map;
norms of residuals are given by (9.575262565796203e-14,).
), time = 7.384228, bytes = 2296348496, gctime = 0.03163975, gcstats = Base.GC_Diff(2296348496, 311, 0, 16758, 12, 274, 31639750, 6, 1))

Check correctness. **You will lose all 20 points if the following statement throws `AssertError`.**

In [29]:
@assert abs(Real(vals[1])) < 1e-8

In [30]:
x_arnoldi   = abs.(Real.(vecs[1]))
x_arnoldi ./= sum(x_arnoldi)
@assert norm(x_arnoldi - xsol) < 1e-8

Arnoldi should be reasonably fast. The points you'll get is

In [31]:
clamp(20 / time_arnoldi * 20, 0, 20)

20.0

**Hint**: My runtime is about 6-7 seconds.

## Q6 (5 pts) Results

List the top 20 pages you found and their corresponding PageRank score. Do they match the top 20 pages ranked according to in-degrees? 

### Solution

In [35]:
score_arnoldi = Dict(Vector(1 : size(A, 2)) .=> x_arnoldi)
print("Top 20 pages and their corresponding PageRank score by Arnoldi method")
first(sort(score_arnoldi, byvalue = true, rev = true), 20)

Top 20 pages and their corresponding PageRank score by Arnoldi method

20-element Vector{Pair{Int64, Float64}}:
 597622 => 0.000914581211451826
  41910 => 0.0009120131809986048
 163076 => 0.0008950559016075303
 537040 => 0.0008899344804414319
 384667 => 0.000779103179016389
 504141 => 0.0007575423485846774
 486981 => 0.0007177642925774769
 605857 => 0.0007108483954459616
  32164 => 0.0007055182681589507
 558792 => 0.0007021658710708856
 551830 => 0.000695075125644683
 765335 => 0.0006762276025731607
 751385 => 0.0006546558408067959
 425771 => 0.0006168480316346489
 908352 => 0.0006146220814775531
 173977 => 0.0006031151911310488
   7315 => 0.00059266429091317
 213433 => 0.0005894474426054685
 885606 => 0.0005812660189536154
 819224 => 0.0005765189744542847

In [36]:
score_sol = Dict(Vector(1 : size(A, 2)) .=> xsol)
print("Top 20 pages and their corresponding PageRank score from the imported solution file")
first(sort(score_sol, byvalue = true, rev = true), 20)

Top 20 pages and their corresponding PageRank score from the imported solution file

20-element Vector{Pair{Int64, Float64}}:
 597622 => 0.000914581211451832
  41910 => 0.0009120131809986237
 163076 => 0.0008950559016075205
 537040 => 0.0008899344804414392
 384667 => 0.000779103179016397
 504141 => 0.0007575423485846804
 486981 => 0.0007177642925774824
 605857 => 0.0007108483954459598
  32164 => 0.0007055182681589599
 558792 => 0.0007021658710709022
 551830 => 0.0006950751256446774
 765335 => 0.0006762276025731783
 751385 => 0.0006546558408067887
 425771 => 0.0006168480316346568
 908352 => 0.0006146220814775524
 173977 => 0.000603115191131058
   7315 => 0.000592664290913173
 213433 => 0.0005894474426054783
 885606 => 0.0005812660189536027
 819224 => 0.0005765189744542846

In [34]:
r_in_sort = sortperm(r_in, dims = 2, rev = true)
print("Top 20 pages ranked according to in-degrees")
collect(zip(r_in_sort, r_in[r_in_sort]))[1:20]

Top 20 pages ranked according to in-degrees

20-element Vector{Tuple{Int64, Int64}}:
 (537040, 6326)
 (597622, 5354)
 (504141, 5271)
 (751385, 5182)
 (32164, 5097)
 (885606, 4847)
 (163076, 4731)
 (819224, 4620)
 (605857, 4550)
 (828964, 4484)
 (551830, 4220)
 (41910, 4219)
 (558792, 4206)
 (459075, 4187)
 (407611, 4180)
 (213433, 4084)
 (765335, 4015)
 (384667, 4010)
 (173977, 3988)
 (687326, 3956)

From above results, it is obvious that the top 20 pages of these two categories are different. This difference implies that the PageRank score is not solely dependent on the in-degree score.

## Q7 Be proud of yourself

Go to your resume/cv and claim you have experience performing analysis on a network of one million nodes.